In [ ]:
#Importing the libraries
import math,copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
#Bringing dataset in desired form and defining ultravariables
my_data = pd.read_csv('classification_train.csv') #read the data
x = my_data.iloc[:,2:]/255
X_train=x.to_numpy()
Y = my_data.iloc[:,1]
y_train=Y.to_numpy()
w_init=np.zeros(len(X_train[0]))
b_init=0.0
print(X_train, y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [8 4 1 ... 9 4 6]


In [ ]:
#Bringing the test dataset in desired form
my_data1 = pd.read_csv('classification_test.csv') #read the data
x1 = my_data1.iloc[:,1:]/255
X_train1=x1.to_numpy()
x2=my_data1.iloc[:,0]
ids=x2.to_numpy()
ids

array([25672, 59964, 66230, ..., 99226, 58519, 80458])

In [ ]:
#creating a list #en with labels for specific class wise binary classification
m=len(X_train)
#en is an array which has separate only 1-0 wala label for every class
p=0
en=np.zeros((10,len(X_train)))
for i in y_train:
  for j in np.unique(y_train):
    if i==j:
      en[j][p]+=1
  p+=1

In [ ]:
#Creating the sigmoid function
def sigmoid(z):
  g=1/(1+np.exp(-z))
  return g

In [ ]:
#Predicting the label
fwb=[]
m=len(X_train)
for i in range(m):
    yp=sigmoid(np.dot(X_train[i], w_init)+b_init)
    fwb.append(yp)
        
fwbs=np.array(fwb)


In [ ]:
def compute_cost(X, y, w, b, *argv):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns:
      total_cost : (scalar) cost 
    """

    m, n = X.shape
    
    
    fwb=[]
    for i in range(m):
        yp=sigmoid(np.dot(X[i], w)+b)
        fwb.append(yp)
        
    fwbs=np.array(fwb)
    
    cost=[]
    loss=0
    for i in range(m):
        loss=-y[i]*np.log(fwbs[i])-(1-y[i])*np.log(1-fwbs[i])
        cost.append(loss)
        
    c=np.array(cost)
    total_cost=np.sum(c)/m 

    return total_cost


In [ ]:
# m, n = X_train.shape

# # Compute and display cost with w and b initialized to zeros
# initial_w = np.zeros(len(X_train[0]))
# initial_b = 0.
# cost = compute_cost(X_train, y_train, initial_w, initial_b)
# print('Cost at initial w and b (zeros): {:.3f}'.format(cost))

In [ ]:
# # Compute and display cost with non-zero w and b
# np.random.seed(1)
# test_w = 0.01 * (np.random.rand(784) - 0.5)
# test_b = -24.
# cost = compute_cost(X_train, en[0], test_w, test_b)

# print('Cost at test w and b (non-zeros): {:.3f}'.format(cost))



In [ ]:

def compute_gradient(X, y, w, b, *argv): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    djn=np.zeros(1)

    ### START CODE HERE ### 
    fwb=[]
    for i in range(m):
        yp=sigmoid(np.dot(X[i], w)+b)
        fwb.append(yp)
        
    fwbs=np.array(fwb)
    
    for j in range(n):    
        for i in range(m):
            dj_dw[j]+=(fwbs[i]-y[i])*X[i][j]
    dj_dw=dj_dw/m
    
    dj_db=(np.sum(fwbs-y))/m
            
    
    ### END CODE HERE ###

        
    return dj_db, dj_dw

In [ ]:
# # Compute and display gradient with w and b initialized to zeros
# initial_w = np.zeros(len(X_train[0]))
# initial_b = 0.

# dj_db, dj_dw = compute_gradient(X_train, y_train, initial_w, initial_b)
# print(f'dj_db at initial w and b (zeros):{dj_db}' )
# print(f'dj_dw at initial w and b (zeros):{dj_dw.tolist()}' )

In [ ]:
# # Compute and display cost and gradient with non-zero w and b
# np.random.seed(1)
# test_w = 0.01 * (np.random.rand(784) - 0.5)
# test_b = -24
# dj_db, dj_dw  = compute_gradient(X_train, y_train, test_w, test_b)

# print('dj_db at test w and b:', dj_db)
# print('dj_dw at test w and b:', dj_dw.tolist())


In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)              Initial value of parameter of the model
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      lambda_ : (scalar, float)    regularization constant
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/2) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [ ]:
#Initialising ultravariables
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(784) - 0.5)
initial_b = -8

In [15]:
#Making the prediction matrix.
pred=np.zeros((10,len(X_train1)))
for i in range(10):
  iterations =50
  alpha = 0.2
  w,b, J_history,_ = gradient_descent(X_train ,en[i], initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations, 0)
  
  #Creating the probability matrix for each class
  fwb=[]
  m=len(X_train1)
  for j in range(m):
    yp=sigmoid(np.dot(X_train1[j], w)+b)
    fwb.append(yp)
        
  fwbs=np.array(fwb)
  pred[i]=fwbs


Iteration    0: Cost     0.53   
Iteration   25: Cost     0.21   
Iteration   49: Cost     0.19   
Iteration    0: Cost     0.59   
Iteration   25: Cost     0.10   
Iteration   49: Cost     0.08   
Iteration    0: Cost     0.48   
Iteration   25: Cost     0.27   
Iteration   49: Cost     0.24   
Iteration    0: Cost     0.57   
Iteration   25: Cost     0.20   
Iteration   49: Cost     0.18   
Iteration    0: Cost     0.44   
Iteration   25: Cost     0.23   
Iteration   49: Cost     0.19   
Iteration    0: Cost     0.74   
Iteration   25: Cost     0.45   
Iteration   49: Cost     0.39   
Iteration    0: Cost     0.54   
Iteration   25: Cost     0.38   
Iteration   49: Cost     0.36   
Iteration    0: Cost     0.67   
Iteration   25: Cost     0.14   
Iteration   49: Cost     0.12   
Iteration    0: Cost     0.52   
Iteration   25: Cost     0.15   
Iteration   49: Cost     0.12   
Iteration    0: Cost     0.53   
Iteration   25: Cost     0.08   
Iteration   49: Cost     0.07   


In [16]:
#final prediction
predict=pred.transpose()
y_final=[]

for i in predict:
  l=0
  for m in range(10):    
    if i[l]==max(i):
      y_final.append(l)
      break
    else:
      l+=1



In [17]:
from numpy import asarray
from numpy import savetxt
savetxt('classification_test(1).csv', y_final, delimiter=',', header='Labels')

In [19]:
my_data = pd.read_csv('classification_test.csv')
my_data

,# Labels
0,0.0
1,1.0
2,2.0
3,0.0
4,1.0
...,...
9995,0.0
9996,6.0
9997,8.0
9998,6.0
